In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os, glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping
from tensorflow.keras.applications import ResNet50, InceptionResNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.metrics import classification_report
from PIL import Image


In [ ]:
metadata = pd.read_csv('/content/drive/MyDrive/GDSC 23-24/Data/metadata.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train, test = train_test_split(metadata, test_size = 0.20, random_state = 42) # double check data variable

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_gen = train_datagen.flow_from_dataframe(
    dataframe = train,
    x_col='filepath',
    y_col='diagnostic',
    target_size=(100, 100),
    class_mode = 'categorical',
    batch_size=32,
    shuffle=True,
    seed=42
)

valid_gen = train_datagen.flow_from_dataframe(
    dataframe = test,
    x_col='filepath',
    y_col='diagnostic',
    target_size=(100, 100),
    class_mode = 'categorical',
    batch_size=32,
    shuffle=False,
    seed=42
)

test_gen = train_datagen.flow_from_dataframe(
    dataframe = test,
    x_col='filepath',
    y_col='diagnostic',
    target_size=(100, 100),
    class_mode = 'categorical',
    batch_size=32,
    shuffle=False
)

Found 1826 validated image filenames belonging to 6 classes.
Found 457 validated image filenames belonging to 6 classes.
Found 457 validated image filenames belonging to 6 classes.


In [ ]:
pretrained_model = ResNet50(
    input_shape=(100,100,3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

inputs = pretrained_model.input
x = Dense(128, activation='relu')(pretrained_model.output)
x = Dense(128, activation='relu')(x)

outputs = Dense(6, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

my_callbacks = [EarlyStopping(monitor = 'val_accuracy',
                              min_delta=0,
                              patience=2,
                              mode='auto')]

history = model.fit(
    train_gen,
    validation_data = valid_gen,
    epochs=100,
)

In [ ]:
# metadata.to_csv('/content/drive/MyDrive/GDSC 23-24/Data/metadata.csv')

In [ ]:
model.save("model_resnet50.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
pretrained_model_2 = ResNet50(
    input_shape=(100,100,3),
    include_top=False,
    weights='imagenet',
)

pretrained_model_2.trainable = False

inputs = pretrained_model_2.input
x = pretrained_model_2.output
x = Conv2D(128, (3, 3), activation='relu')(x)
# x = MaxPooling2D(pool_size=(2,2))(x)
# x = Conv2D(32, (1, 1), activation='relu')(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

outputs = Dense(6, activation='softmax')(x)
model_2 = Model(inputs=inputs, outputs=outputs)

model_2.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

my_callbacks = [EarlyStopping(monitor = 'val_accuracy',
                              min_delta=0,
                              patience=2,
                              mode='auto')]

history = model_2.fit(
    train_gen,
    validation_data = valid_gen,
    epochs=100,
)

Epoch 1/100
58/58 [==============================] - 213s 3s/step - loss: 2.0661 - accuracy: 0.4606 - val_loss: 1.1618 - val_accuracy: 0.6083
Epoch 2/100
58/58 [==============================] - 193s 3s/step - loss: 1.0968 - accuracy: 0.6035 - val_loss: 1.0936 - val_accuracy: 0.6018
Epoch 3/100
58/58 [==============================] - 193s 3s/step - loss: 0.9026 - accuracy: 0.6802 - val_loss: 1.0713 - val_accuracy: 0.6346
Epoch 4/100
58/58 [==============================] - 198s 3s/step - loss: 0.7037 - accuracy: 0.7415 - val_loss: 1.1158 - val_accuracy: 0.6193
Epoch 5/100
58/58 [==============================] - 197s 3s/step - loss: 0.6092 - accuracy: 0.7694 - val_loss: 1.2004 - val_accuracy: 0.6324
Epoch 6/100
58/58 [==============================] - 197s 3s/step - loss: 0.4588 - accuracy: 0.8335 - val_loss: 1.2770 - val_accuracy: 0.6324
Epoch 7/100
58/58 [==============================] - 204s 4s/step - loss: 0.3672 - accuracy: 0.8614 - val_loss: 1.3766 - val_accuracy: 0.5952
Epoch 

In [ ]:
model_2.save("model_2_resnet50.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
results = model_2.evaluate(test_gen, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
#print("Test Accuracy: {:.2f}%".format(results[1] * 100))

    Test Loss: 4.46033
Test Accuracy: 65.21%


In [ ]:
pretrained_model_3 = ResNet50(
    input_shape=(100,100,3),
    include_top=False,
    weights='imagenet',
)

pretrained_model_3.trainable = False

inputs = pretrained_model_3.input
x = pretrained_model_3.output
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
#x = MaxPooling2D(pool_size=(2,2))(x)

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
#x = MaxPooling2D(pool_size=(2,2))(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)

x = Flatten()(x)

x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)

x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

outputs = Dense(6, activation='softmax')(x)
model_3 = Model(inputs=inputs, outputs=outputs)

model_3.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

my_callbacks = [EarlyStopping(monitor = 'val_accuracy',
                              min_delta=0,
                              patience=2,
                              mode='auto')]

history = model_3.fit(
    train_gen,
    validation_data = valid_gen,
    epochs=100,
)

Epoch 1/100
58/58 [==============================] - 180s 3s/step - loss: 1.4168 - accuracy: 0.4748 - val_loss: 1.2407 - val_accuracy: 0.5646
Epoch 2/100
58/58 [==============================] - 185s 3s/step - loss: 1.1399 - accuracy: 0.5997 - val_loss: 1.1094 - val_accuracy: 0.6061
Epoch 3/100
58/58 [==============================] - 176s 3s/step - loss: 0.9562 - accuracy: 0.6462 - val_loss: 1.0404 - val_accuracy: 0.6389
Epoch 4/100
58/58 [==============================] - 171s 3s/step - loss: 0.8599 - accuracy: 0.6950 - val_loss: 1.0587 - val_accuracy: 0.6368
Epoch 5/100
58/58 [==============================] - 177s 3s/step - loss: 0.7333 - accuracy: 0.7393 - val_loss: 1.2037 - val_accuracy: 0.6433
Epoch 6/100
58/58 [==============================] - 171s 3s/step - loss: 0.6865 - accuracy: 0.7579 - val_loss: 1.1742 - val_accuracy: 0.6674
Epoch 7/100
58/58 [==============================] - 171s 3s/step - loss: 0.5089 - accuracy: 0.8100 - val_loss: 1.7876 - val_accuracy: 0.6586
Epoch 

In [ ]:
model_5.save("model_5_resnet50.h5")